# Predicting Bone Cancer Model VGG16 and VGG19

    using transfer learning

# Model Train
    

In [1]:
# import the libraries as shown below
# we are using VGG16 and VGG19
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import time
print(tf.__version__) #here check the tf version

2.3.0


In [2]:
#tensorboard model name with time function
#make a folder named logs
CANCER = "predict-bone-cancer-cnn-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(CANCER))

In [3]:
# resize all train images
IMAGE_SIZE = [224, 224]

# we are train our model with 84 images
# we are providing here two folder for training 

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we are providing the "IMAGE_SIZE" and also the RGB dimenssion 3
# Here we will be using imagenet weights for categorises the images
# Basically VGG16 used for classify thousand different categorises  of images that's why "include_top = False". 
# Because we are training 2 categorises images. So we are droping the last column
# Also droping the first column because we are providing the "IMAGE_SIZE"

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
# Putting the for loop and make sure that all the layer shouldn't be train

for layer in vgg.layers:
    layer.trainable = False

In [6]:
# how many classes or categorises in my train dataset
folders = glob('Datasets/train/*')

In [7]:
# After the droping the first & last layer
# We just the the Flatten layer
# Flatten layer basically converts the features map to a single column

x = Flatten()(vgg.output)

In [8]:
# Finally adding the last layer(Dense) and also provide the folder length. We need how many categories we have.
# Then we are using the activation function called "softmax"

prediction = Dense(len(folders), activation='softmax')(x)


# Then combine the vgg output and input this will create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
# view  the model strcture 
# Absorve the last layer(Dense) it is just having 2 output categorises
# Because our dataset  have 2 categorises 

model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

# Model Fit & A File Create For Loading the model
    

In [10]:
# tell the model what cost and optimization method to use
# Adam optimizer provide an optimization algorithm which can handle sparse gradients on noisy problems.

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [11]:
# Use the Image Data Generator to import the images from the dataset
# we upload dataset using ImageDataGenerator which will help in train and test dataset.

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# we insert images using flow_from_directory . For batch size, at a time 32 images will demand for training


training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 45 images belonging to 2 classes.


In [13]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 39 images belonging to 2 classes.


In [14]:
# fit the model
# we use fit_generator from library for running it. we will use train data set for running and test data for validation.
#tensorboard callbacks function for visualize accuracy and loss graph

r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=8,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks = [tensorboard]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/8
1/2 [==============>...............] - ETA: 0s - loss: 0.6811 - accuracy: 0.5385WARNING:tensorflow:From C:\Users\hasib\.conda\envs\thesis21\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2/2 [==============================] - 20s 10s/step - loss: 1.2976 - accuracy: 0.6889 - val_loss: 1.9194 - val_accuracy: 0.7436
Epoch 2/8
2/2 [==============================] - 15s 8s/step - loss: 1.2620 - accuracy: 0.7778 - val_loss: 0.8712 - val_accuracy: 0.7436
Epoch 3/8
2/2 [==============================] - 15s 7s/step - loss: 0.4124 - accuracy: 0.7556 - val_loss: 1.1163 - val_accuracy: 0.4615
Epoch 4/8
2/2 [==============================] - 15s 8s/step - loss: 0.5536 - accuracy: 0.6889 - val_loss: 0.8038 - val_accuracy: 0.7692
Ep

In [15]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('our-model.h5')

# Predicting based on the model
        
    1. Import all the necessary libraries we need available in keras
    6. Load the model vgg19 to categorize thousand of different categories.
    7. load image set we have use for the project,we load training path. Here we set target size as 224,224 because here        we are working in RGB type of images as it is 3D.
    8. Convert to numpy array
    9. Expand the shape of the array. A new axis will appear at the axis position in the expanded array shape.
    10. Preprocessing input x
    11. It returns the probability for each outcome class as a value between 0 and 1.
    12. set classes for malignant
    13. set classes for normal
    15. Set a condition if malignant print malignant. Otherwise print normal.

In [16]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
import numpy as np
model=load_model('our-model.h5')
img=image.load_img('Datasets/val/malignant/m (5).JPG', target_size=(224, 224))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
classes=model.predict(img_data)
malignant=classes[0,0]
normal=classes[0,1]

if(malignant==1):
    print('P: malignant')
else:
    print('P: normal')


P: malignant
